In [2]:
import numpy as np
import pandas as pd
import os
import zipfile
import requests
import json 
from bs4 import BeautifulSoup 
import boto3 
import time
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [3]:
url = 'https://ies-fsv.s3.eu-central-1.amazonaws.com/studentsSets/45247808.zip'
response = requests.get(url)

with open('data.zip', 'wb') as file:
    file.write(response.content)

In [4]:
zip_file_path = 'data.zip'
with zipfile.ZipFile(zip_file_path, 'r') as z:
    print(z.namelist())

['data/MU.csv', 'data/MCD.csv', 'data/LIN.csv', 'data/DXCM.csv']
